In [ ]:
import os
import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications import *
import numpy as np
import pandas as pd

In [ ]:
!unzip "/content/drive/My Drive/Venomous Snake Classifier/train.zip" -d "/content/train"
!unzip "/content/drive/My Drive/Venomous Snake Classifier/test.zip" -d "/content/test"
!unzip "/content/drive/My Drive/Venomous Snake Classifier/val.zip" -d "/content/val"

Streaming output truncated to the last 5000 lines.
  inflating: /content/val/content/data/val/venomous/4e08245f699458b4026d1c1762f65d8f.jpg  
  inflating: /content/val/content/data/val/venomous/b0257e8c8e1a79c89df55b8ca03659c9.jpg  
  inflating: /content/val/content/data/val/venomous/16990da16c2733af5628f785b4331ab9.jpg  
  inflating: /content/val/content/data/val/venomous/44f8b730c809623e4d9ede95d7f8c847.jpg  
  inflating: /content/val/content/data/val/venomous/1b4a26cf88012ac4d9667cb4a2de41d2.jpg  
  inflating: /content/val/content/data/val/venomous/33d6d1ee1cc18b861321f64cba034128.jpg  
  inflating: /content/val/content/data/val/venomous/e2242c8ca35818e3dcfc03eaebc9cbba.jpg  
  inflating: /content/val/content/data/val/venomous/055843c0358c6bcd283a448c62b617b5.jpg  
  inflating: /content/val/content/data/val/venomous/5732d166a84a0174df228dfab3a74d83.jpg  
  inflating: /content/val/content/data/val/venomous/4ebf936d036bb08ca2f7bfd133f6ea6f.jpg  
  inflating: /content/val/content/data/

In [ ]:
import shutil

# original = r'/content/train/content/data/train'
# target = r'/content/Train'

# shutil.move(original,target)

# original = r'/content/test/content/data/test'
# target = r'/content/Test'

# shutil.move(original,target)


# original = r'/content/val/content/data/val'
# target = r'/content/Val'

# shutil.move(original,target)

# !rm -r '/content/test'
# !rm -r '/content/val'
# !rm -r '/content/train'

In [ ]:
nb_classes = 2
img_width, img_height = 224, 224
batch_size = 32 
nb_epoch = 50
learn_rate = 1e-4
transformation_ratio = .05
train_dir = '/content/train'
validation_dir = '/content/val'

base_model = Xception(input_shape=(img_width, img_height, 3), weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(nb_classes, activation='softmax')(x)

model = Model(base_model.input, predictions)
print(model.summary())

for layer in base_model.layers:
    layer.trainable = False

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')
    

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=(img_width, img_height),
                                                                  batch_size=batch_size,
                                                                  class_mode='categorical')

model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
callbacks_list = [
        ModelCheckpoint(top_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True),
        EarlyStopping(monitor='val_accuracy', patience=5, verbose=0)
    ]

model.fit_generator(train_generator,
                        steps_per_epoch=200,
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        callbacks=callbacks_list)

model.load_weights(top_weights_path)

for layer in model.layers[:based_model_last_block_layer_number]:
        layer.trainable = False
for layer in model.layers[based_model_last_block_layer_number:]:
        layer.trainable = True

model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
callbacks_list = [
        ModelCheckpoint(final_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5, verbose=0)
    ]

model.fit_generator(train_generator,
                        steps_per_epoch=200,
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        callbacks=callbacks_list)

model_json = model.to_json()
with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
        json_file.write(model_json)



Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [ ]:
pd.read_csv('/content/sub.csv')

In [ ]:
import shutil, os
shutil.move('/content/test', '/content/Test')

'/content/Test/test'

In [ ]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("/content/model_weights.h5")
test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    directory='/content/Test/',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
)
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

Found 14927 images belonging to 1 classes.
14927/14927 [==============================] - 267s 18ms/step


In [ ]:
yo = pd.read_csv('/content/sub.csv')
arr = yo.id.values

In [ ]:
print(arr[:5])

['f285af73cb3f70601abf5be31' '48bcd3e229f4fc85aa590df2f'
 'd4c46a34dc0a0dbba228579b5' 'c483c79711c00af486b535143'
 '482104b039372896b6b5731d7']


In [ ]:
ar=[]
for files in os.walk(r'/content/Test/test'):
    for filename in files:
      ar.append(filename)
print(ar)
if '00022e97c90898c02a25579be.jpg' in ar:
  print(list(ar).index('00022e97c90898c02a25579be.jpg'))
else:
  print('nolo')
# for i,j in zip(ar,arr):
#   if i!=j:
#     print('yolo')

['/content/Test/test', [], ['f285af73cb3f70601abf5be31.jpg', '48bcd3e229f4fc85aa590df2f.jpg', 'd4c46a34dc0a0dbba228579b5.jpg', 'c483c79711c00af486b535143.jpg', '482104b039372896b6b5731d7.jpg', '45a0a2f5c4616e51e69eeb21a.jpg', '7abdd9dc82f0944c51296a4b4.jpg', 'ff108760431ebb5e0d746bf80.jpg', '7d436cf9d97579796f560ca45.jpg', 'd59363f5e3cf189322ea7dd79.jpg', 'a023e0cad3816339f8d882647.jpg', 'd8cb98ff289923424a9089dcd.jpg', '9eac47fb45c6d678a03ec81dc.jpg', '77b6056deba4d7b8c05e03e9d.jpg', 'a7be72f11dbf6d6b3b4837193.jpg', '58520485a2660620c8fc477b7.jpg', '436815a5e8553aecf719b3e67.jpg', 'e81dddcdff8f88c5112322237.jpg', '7b58d265e8313a7e4dc493a68.jpg', '8c3072b17ecdb44ed6a65923b.jpg', '5a192372665217fae6b65f359.jpg', '6590117f948ee9a295cbd83aa.jpg', 'b9352d559141cc7919a6877cf.jpg', 'e1cf9c309482e71ddd48f02a9.jpg', '615af36e141395a79833aea33.jpg', '5f98edd965d2237f708436fff.jpg', '20e2e1845f9a521b014c120ac.jpg', 'cc80043fedad15254b461057d.jpg', '7f662c9dbcde6ce553a5446e4.jpg', '6a4fc18523c309

In [ ]:
final=[]
for i in pred:
  if i[0]>i[1]:
    final.append('non_venomous')
  else:
    final.append('venomous')

In [ ]:

l = pd.read_csv('/content/drive/My Drive/Venomous Snake Classifier/sample_submission.csv')
arr=l.id.values

In [ ]:
print(final)

['non_venomous', 'venomous', 'venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'venomous', 'venomous', 'venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'venomous', 'venomous', 'venomous', 'non_venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'venomous', 'venomous', 'non_venomous', 'non_venomous', 'venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'venomous', 'venomous', 'non_venomous', 'venomous', 'venomous', 'non_venomous', 'venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'non_venomous', 'venomous', 'no

In [ ]:
print(arr[:5])

['00022e97c90898c02a25579be' '000816b96db153cc47dfb79db'
 '001d2fe5ca243c138472d130b' '001e3329abba16fb91ee4d47f'
 '0026cf1ed9556341340419b19']


In [ ]:
temp = {'id':arr,'class':final}
sub = pd.DataFrame(temp)
submit = sub.to_csv('/content/sub.csv',index=False)

In [ ]:
()